In [58]:
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd

from collections import defaultdict
from pymongo import MongoClient
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
client = MongoClient('mongodb://52.90.19.86:27017')
db = client.travel_recommender
collection = db.wiki_voyage

In [16]:
tree = ET.parse('/Users/cgraham/Development/metis_work/projects/04_Fletcher/wikivoyage_edited.xml')
root = tree.getroot()
pages = root.findall('page')

In [17]:
collection_list = []

for page in pages:
    title = page.find('title').text
    text = page.find('revision/text').text

    if (title != '' and 'Wikivoyage' not in title and 'MediaWiki' not in title and 'File:' not in title and 
        'Template:' not in title and 'Category:' not in title and text != '' and '#REDIRECT' not in text):
        doc = {"city": title, "text": text}
        collection_list.append(doc)

documents = [row.get('text') for row in collection_list]
# results = collection.insert_many(collection_list)

28105

In [22]:
vectorizer = CountVectorizer(ngram_range=(1,2), stop_words='english', token_pattern='\\b[a-z][a-z]+\\b', min_df=0.1, max_df=.90)
myvector = vectorizer.fit_transform(documents)
words = sorted([(i,v) for v,i in vectorizer.vocabulary_.items()])

In [82]:
vectorizer.get_feature_names()

[u'access',
 u'accessible',
 u'accommodation',
 u'activities',
 u'address',
 u'address lat',
 u'africa',
 u'air',
 u'airport',
 u'alt',
 u'alt url',
 u'america',
 u'american',
 u'ancient',
 u'approximately',
 u'april',
 u'area',
 u'areas',
 u'arrive',
 u'art',
 u'ask',
 u'atmosphere',
 u'attractions',
 u'august',
 u'available',
 u'ave',
 u'away',
 u'banner',
 u'banner jpg',
 u'bar',
 u'bars',
 u'base',
 u'based',
 u'bay',
 u'beach',
 u'beaches',
 u'beautiful',
 u'bed',
 u'beer',
 u'best',
 u'better',
 u'big',
 u'bike',
 u'bit',
 u'black',
 u'blue',
 u'boat',
 u'book',
 u'border',
 u'breakfast',
 u'bridge',
 u'bring',
 u'budget',
 u'building',
 u'buildings',
 u'built',
 u'bus',
 u'bus station',
 u'buses',
 u'business',
 u'buy',
 u'buy eat',
 u'cafe',
 u'called',
 u'camping',
 u'capital',
 u'car',
 u'castle',
 u'category',
 u'center',
 u'central',
 u'centre',
 u'century',
 u'change',
 u'cheap',
 u'check',
 u'checkin',
 u'checkin checkout',
 u'checkout',
 u'checkout content',
 u'checkout 

In [84]:
model = NMF(n_components=20, init='random', random_state=0)
doc_topics = model.fit_transform(myvector)

In [73]:
print doc_topics.shape
doc_topics[0]

(28105, 5)


array([ 5.5109521 ,  2.71841163,  3.87986925,  0.92432642,  2.01639297])

In [69]:
doc_cluster = [list(r).index(max(r)) for r in doc_topics]
clusters = defaultdict(list)

for i, d in enumerate(doc_cluster):
    clusters[d].append(i)

In [70]:
model.components_.shape

(5, 635)

In [74]:
topic_words = []
for r in model.components_:
    a = sorted([(v,i) for i,v in enumerate(r)],reverse=True)[0:7]
    topic_words.append([words[e[1]] for e in a])

In [78]:
topic_words

[[(253, u'hours'),
  (190, u'fax hours'),
  (563, u'tollfree'),
  (564, u'tollfree fax'),
  (414, u'phone tollfree'),
  (188, u'fax'),
  (325, u'long')],
 [(85, u'city'),
  (56, u'bus'),
  (260, u'http'),
  (516, u'south'),
  (369, u'new'),
  (402, u'park'),
  (631, u'www')],
 [(9, u'alt'),
  (10, u'alt url'),
  (430, u'price'),
  (4, u'address'),
  (108, u'content'),
  (413, u'phone'),
  (137, u'directions')],
 [(325, u'long'),
  (300, u'lat'),
  (301, u'lat long'),
  (264, u'image'),
  (9, u'alt'),
  (108, u'content'),
  (137, u'directions')],
 [(260, u'http'),
  (631, u'www'),
  (261, u'http www'),
  (595, u'url http'),
  (95, u'com'),
  (9, u'alt'),
  (10, u'alt url')]]